In [3]:
import pandas as pd
import Resources.utils_stripped as U

In [5]:
df = pd.read_csv("Input/opp115_train.csv")  # moet Sentence + target bevatten, label is optioneel
train_df, test_df = U.load_opp115_from_df(df)     # test_df kan None zijn als dataset_type ontbreekt
df_train, df_val = U.split_train_val(train_df, val_size=0.2)

# Training of Models

## RoBERTa Model

In [6]:
res = U.train_transformer_mcc(
    model_type="roberta",
    df_train=df_train,
    df_val=df_val,
    epochs=3,
    batch_size=32,
    lr=2e-5,
    out_dir="artifacts/roberta"
)

model_roberta = res["model"]
tokenizer_roberta = res["tokenizer"]

# # inference
# pred_labels, pred_proba = U.predict_transformer(model_roberta, tokenizer_roberta, ["Some privacy sentence..."], return_proba=True)
# pred_labels[0]

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


KeyboardInterrupt: 

## BERT Model

In [7]:
# 3) Train BERT (multi-class)
res = U.train_transformer_mcc(
    model_type="bert",
    df_train=df_train,
    df_val=df_val,
    epochs=3,
    batch_size=32,
    lr=2e-5,
    out_dir="artifacts/bert_mcc"
)

bert_model = res["model"]
bert_tokenizer = res["tokenizer"]

# Inference
# pred_labels, pred_proba = U.predict_transformer(bert_model, bert_tokenizer, ["Some privacy sentence..."], return_proba=True)
# pred_labels[0]


KeyboardInterrupt: 

## BiLSTM Model

In [8]:
df_train, df_val = U.split_train_val(train_df, val_size=0.2)

res = U.train_bilstm_mcc(
    df_train=df_train,
    df_val=df_val,
    epochs=3,
    batch_size=32,
    lr=1e-4,
    out_dir="artifacts/bilstm"
)

bilstm = res["model"]

# # inference
# pred_labels, pred_proba = U.predict_bilstm(bilstm, ["Some privacy sentence..."], return_proba=True)
# pred_labels[0]


Batches: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 76/76 [00:10<00:00,  7.25it/s]
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallel